In [ ]:
import os.path
import time

from Bio import SeqIO
import h5py
import numpy as np

In [ ]:
input_fp = '/home/jklynch/host/project/viral-learning/data/500_ArcPhage_training_set.fasta'

In [ ]:
for record in SeqIO.parse(input_fp, "fasta"):
    print(record.id)
    print(record.seq)
    break


## make a 2D sequence

From a sequence of N base pairs like (N=10)
```
ATGAGTTCTG
```
create a 2D sequence with height N-M+1 and width M like this (N=10, M=5)
```
  ATGAG
  TGAGT
  GAGTT
  AGTTC
  GTTCT
  TTCTG
```

Start by generating start:stop indices like this:
```
  0:5  (0,5) + 0
  1:6  (0,5) + 1
  2:7  (0,5) + 2
  ...
```

Use a numpy array
```
start_stop = np.zeros((6,2))
           -> [[0 0]
               [0 0]
               [0 0]
               [0 0]
               [0 0]
               [0 0]]
start_stop[:, 1] = 5
start_stop -> [[0 5]
               [0 5]
               [0 5]
               [0 5]
               [0 5]
               [0 5]]
start_stop = start_stop + np.arange(start_stop.shape[1]).reshape(5,1)
start_stop -> [[0 5]
               [1 6]
               [2 7]
               [3 8]
               [4 9]
               [5 10]]
```

In [ ]:
def get_start_stop(N, M):
    #N = 10
    #M = 5
    start_stop = np.zeros((N-M+1,2), dtype=np.int64)
    start_stop[:, 1] = M
    start_stop = start_stop + np.arange(N-M+1, dtype=np.int64).reshape((N-M+1,1))
    return start_stop

In [ ]:
get_start_stop(N=11, M=10)

Use get_start_stop() to generate the 2D sequence.

In [ ]:
def get_2D_sequence(seq, M):
    seq_2d = []
    for start, stop in get_start_stop(N=len(seq), M=M):
        seq_2d.append(seq[start:stop])
    return seq_2d

In [ ]:
seq_2D = get_2D_sequence(str(record.seq[0:11]), M=10)
print('seq_2d len: {}'.format(len(seq_2D)))
seq_2D

## generate numerical data

Use 5 channels, one for each nucleotide.
```
  d = np.zeros(S, N-M+1, M, 4)
```
where S is the number of samples.

Create a dictionary to map A,C,G,T,U to channels 0,1,2,3,4:
```
  {A:0, C:1, G:2, T:3}
```

In [ ]:
def translate_seq_to_input(seq, M, verbose=False):
    nucleotide_to_channels = {
        'A':[1.00, 0.00, 0.00, 0.00, 0.00],
        'C':[0.00, 1.00, 0.00, 0.00, 0.00],
        'G':[0.00, 0.00, 1.00, 0.00, 0.00],
        'T':[0.00, 0.00, 0.00, 1.00, 0.00],
        'U':[0.00, 0.00, 0.00, 0.00, 0.00],
        'N':[0.20, 0.20, 0.20, 0.20, 0.20],
        'R':[0.50, 0.00, 0.50, 0.00, 0.00],
        'M':[0.50, 0.50, 0.00, 0.00, 0.00], # A or C
        'S':[0.00, 0.50, 0.50, 0.00, 0.00], # C or G
        'K':[0.00, 0.00, 0.333, 0.333, 0.333], # G, T, or U
        'W':[0.333, 0.00, 0.00, 0.333, 0.333], # A, T, or U
        'Y':[0.00, 0.333, 0.00, 0.333, 0.333]} # C, T, ur U
    ##S = 1
    N = len(seq)
    ##M = 100
    input_data = np.zeros((N-M+1, M, 5))
    for start, partial_seq in enumerate(get_2D_sequence(seq, M=M)):
        #print(partial_seq)
        for n, nucleotide in enumerate(partial_seq):
            input_data[start, n, :] = nucleotide_to_channels[nucleotide]
        if verbose:
            print(partial_seq)
            print(input_data[start, :, :])

    return input_data


In [ ]:
S = 1
seq = str(record.seq[0:11])
N = len(seq)
M = 10
seq_2d = get_2D_sequence(seq=seq, M=M)
print('length of seq_2d: {}'.format(len(seq_2d)))
print('seq_2d:\n{}'.format(seq_2d))
d = np.zeros((S, N-M+1, M, 5))
print('d.shape: {}'.format(d.shape))
d[0, :, :, :] = translate_seq_to_input(seq=seq, M=M, verbose=True)

## Write H5 file
Everybody is doing it.

In [ ]:
def write_2d_seq_h5_file(input_seq_fp, output_h5_fp, M):
    with h5py.File(output_h5_fp, 'w') as h5_file:

        # write a dataset to h5_file with these dimensions:
        #   (None, N-M+1, M, 4)
        #
        # where 
        #   None indicates an arbitrary number of sequences
        #   N is the length of each sequence in seq_fp
        #   M is the width of each 2d sequence 'image'

        # read the first sequence to get its length
        sample_count = 0
        for record in SeqIO.parse(input_seq_fp, "fasta"):
            if sample_count == 0:
                first_seq = str(record.seq)
            sample_count += 1
        print('found {} sequences in {}'.format(sample_count, input_seq_fp))

        N = len(first_seq)

        seq_2d_dset = h5_file.create_dataset(
            name=os.path.basename(input_seq_fp),
            shape=(sample_count, N-M+1, M, 5),
            maxshape=(None, N-M+1, M, 5),
            dtype=np.float64,
            chunks=(1, N-M+1, M, 5),
            compression='gzip',
            compression_opts=9)

        for r, record in enumerate(SeqIO.parse(input_seq_fp, "fasta")):
            if len(record.seq) != 500:
                print('{} record.seq length: {}'.format(r, len(record.seq)))
            else:
                t0 = time.time()
                seq_2d_dset[r, :, :, :] = translate_seq_to_input(seq=str(record.seq), M=M)
                print('finished record {} in {:5.2f}s'.format(r, time.time()-t0))


In [ ]:
input_basename, ext = os.path.splitext(os.path.basename(input_fp))
print(input_basename)

write_2d_seq_h5_file(input_seq_fp=input_fp, output_h5_fp=input_basename + '.h5', M=100)